In [4]:
!pip install portalocker>=2.0.0
!pip install transformers
!pip install pytorch_lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 34.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 727.0/727.0 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 763.4/763.4 kB 9.3 MB/s eta 0:00:00


In [5]:
import torch
import math
import torchtext.vocab as Vocab
from torch import nn
from torch.utils.data import DataLoader, Dataset
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# pour monter son drive Google Drive local
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
my_local_drive='/content/gdrive/My Drive/Colab Notebooks/transformers'
import sys
# Ajout du path pour les librairies, fonctions et données
sys.path.append(my_local_drive)
# Se positionner sur le répertoire associé
%cd $my_local_drive


%pwd
%ls

/content/gdrive/My Drive/Colab Notebooks/transformers
'=2.0.0'
 checkpoints/
'Copy of translation_transformer.ipynb'
 dataset/
 eng_-french.csv
'kaggle (1).json'
 kaggle.json
 lightning_logs/
 mroedata_translation_transformer.ipynb
 prepare_data.ipynb
 test_dataset.pth
 train_dataset.pth
 translation_transformer.ipynb
 Ttransformer.ipynb


In [1]:
from google.colab import files
files.upload()  # Upload your kaggle.json file

!pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset
!kaggle datasets download -d dhruvildave/en-fr-translation-dataset
# Unzip the dataset
!unzip en-fr-translation-dataset.zip

Saving kaggle.json to kaggle.json
100% 2.54G/2.54G [01:57<00:00, 24.3MB/s]
100% 2.54G/2.54G [01:57<00:00, 23.2MB/s]
Archive:  en-fr-translation-dataset.zip
  inflating: en-fr.csv               


In [11]:
%ls

'=2.0.0'
 checkpoints/
'Copy of translation_transformer.ipynb'
 dataset/
 en-fr.csv
 eng_-french.csv
'kaggle (1).json'
 kaggle.json
 lightning_logs/
 mroedata_translation_transformer.ipynb
 prepare_data.ipynb
 test_dataset.pth
 train_dataset.pth
 translation_transformer.ipynb
 Ttransformer.ipynb


In [6]:
NEG_INF = -1E10
POSITIONAL_ENCODING_DIVISOR = 10000
batch_size=8

In [14]:
import pandas as pd
from torch.nn.utils.rnn import pad_sequence
from transformers import BertTokenizer, GPT2Tokenizer
import random
import warnings
import torch
from torch.utils.data import Dataset
import numpy as np
from sklearn.model_selection import train_test_split
from tqdm import tqdm

# Read the entire dataset
# Note: Replace this with the actual path to your dataset
df = pd.read_csv("en-fr.csv", nrows=1000000).dropna()

# Perform train-test split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

# Initialize the tokenizers
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
gpt2_tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Data processing pipelines
english_pipeline = lambda x: bert_tokenizer.encode("< " + x + " >", add_special_tokens=False)
french_pipeline = lambda x: gpt2_tokenizer.encode("< " + x + " >", add_special_tokens=False)

# Create the bucketed dataset
class BucketedDataset(Dataset):
    def __init__(self, data, english_pipeline, french_pipeline, num_buckets):
        self.buckets = {}
        self.batch_size = batch_size  # Assuming batch_size is globally defined

        lengths = [max(len(english_pipeline(english)), len(french_pipeline(french)))
                   for english, french in tqdm(zip(data['en'], data['fr']), total=len(data), desc="Calculating lengths")]

        sorted_indices = np.argsort(lengths)
        bucket_size = len(sorted_indices) // num_buckets

        for i in tqdm(range(num_buckets), desc="Filling buckets"):
            start_idx = i * bucket_size
            end_idx = (i + 1) * bucket_size if i != num_buckets - 1 else len(sorted_indices)
            subset_indices = sorted_indices[start_idx:end_idx]

            self.buckets[(start_idx, end_idx)] = [(torch.tensor(english_pipeline(data.iloc[idx]['en']), dtype=torch.int64),
                                                   torch.tensor(french_pipeline(data.iloc[idx]['fr']), dtype=torch.int64))
                                                  for idx in subset_indices]

        for rng, bucket in self.buckets.items():
            if len(bucket) < self.batch_size:
                warnings.warn(f"Bucket {rng} has only {len(bucket)} samples which is less than the batch size of {self.batch_size}. Consider using a different bucketing strategy or oversampling.")

    def __getitem__(self, idx):
        selected_bucket = random.choice(list(self.buckets.keys()))
        return random.sample(self.buckets[selected_bucket], self.batch_size)

    def __len__(self):
        return sum([len(bucket) for bucket in self.buckets.values()]) // self.batch_size

# Global settings
num_buckets = 200  # Number of buckets
batch_size = 8     # Batch size

# Initialize the datasets
print("Creating training dataset...")
train_dataset = BucketedDataset(train_df, english_pipeline, french_pipeline, num_buckets)

print("Creating testing dataset...")
test_dataset = BucketedDataset(test_df, english_pipeline, french_pipeline, num_buckets)

# Save the datasets
torch.save(train_dataset, 'train_dataset_2.pth')
torch.save(test_dataset, 'test_dataset_2.pth')


Creating training dataset...


Streaming output truncated to the last 5000 lines.
Calculating lengths: 100%|██████████| 799985/799985 [31:08<00:00, 428.20it/s]

Filling buckets: 100%|██████████| 200/200 [28:44<00:00,  8.62s/it]


Creating testing dataset...



Calculating lengths: 100%|██████████| 199997/199997 [07:38<00:00, 436.26it/s]

Filling buckets: 100%|██████████| 200/200 [07:12<00:00,  2.16s/it]
